In [409]:
from pyscipopt import Model, quicksum
import numpy as np
from itertools import product
import json

In [410]:
#Importing constant variables
with open('./EVRP_LP/input_variables.json', 'r') as file:
    input_variables = json.load(file)

# print(input_variables["Ctt"])

In [411]:
def instanSacy(n_customers, n_vehicles, max_demand, max_distance):
    No = set(np.arange(1,n_customers+1)) #Set of customers
    N = No | {0} #Customers + depot
    F = {0, "F1", "F2"} #Set of recharging stations
    G = N | F #Full graph
    A = [(i,j) for i,j in product(G,G) if i!=j] #Set of arcs
    
    demand = {i: (0 if i in F else int(np.random.randint(100, max_demand, 1)[0])) for i in G} #Demand per customer

    M = list(np.arange(1,n_vehicles+1)) #Set of vehicles

    load_capacity = {m:input_variables["Qm"] for m in M} #Load_capacity per vehicle
    
    '''Time cost as a function of distance and avg. speed'''
    distance =  {(i,j):int(np.random.randint(50, max_distance, 25)[0]) for i,j in A}
    time_cost = {(i,j):round((distance[i,j] / input_variables["Avg_V"]),2) for i,j in A} #Travel time cost per arc
    # time_cost = {(i,j):int(np.random.randint(1, max_time_cost, 1)[0]) for i,j in A} 
    
    battery_capacity = input_variables["Battery_capacity_Wh"]

    '''Energy cost as a function of Energy Consumption per Km'''
    EV_avg_energy_consumption = (290+150)/2 #in Wh font:'https://ev-database.org/cheatsheet/energy-consumption-electric-car'
    CE_avg_energy_consumption = (0.16) #Liters/km font:'https://www.smmt.co.uk/wp-content/uploads/sites/2/Heavy-CV-Fuel-Consumption-Fact-Sheet.pdf'
    EV_energy_cost = {(i,j):(EV_avg_energy_consumption/distance[i,j]) for i,j in A} #Energy cost per arc
    CE_energy_cost = {(i,j):(CE_avg_energy_consumption/distance[i,j]) for i,j in A} #Fuel cost per arc
    # energy_cost = {(i,j):int(np.random.randint(1, max_energy_cost, 1)[0]) for i,j in A if j>i} #Energy cost per arc

    
    return No, N, F, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost

In [412]:
def sacystation(No, N, F, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost):
    """transp -- model for solving the transportation problem
    Parameters:
        No - set of customers
        N - customers + depot
        F - set of charging stations
        G - N + F
        A - set of arcs (i,j)
        M - set of vehicles
        energy_cost[i,j] - energy transportation cost on arc (i,j)
        demand[i] - demand of customer i
        load_capacity[m] - capacity of vehicle m
        time_cost - Travel time per arc(i,j)

    ==================================
    Problems:
    Dar nome significativo às restrições 
    Adicionar as estações de recarga - u[i]?
    """
    
    model = Model("sacystation")

    #Binary decision variable
    x = {}
    for (i,j,m) in [(i,j,m) for i,j in A for m in M]:
        x[i,j,m] = model.addVar(vtype="B", name=f"x{(i,j,m)}")
        
    #Load Variable
    l = {}
    for (i,j) in [(i,j) for i,j in A]:
        l[i,j] = model.addVar(vtype="C", name=f"l{(i,j)}")
    
    #Remaining Battery Variable
    y = {}
    for (i,j,m) in [(i,j,m) for i,j in A for m in M]:
        y[i,j,m] = model.addVar(vtype="C", name=f"y{(i,j,m)}")
        
    # print(x,L,y, sep='\n')
    
    '''
    Constraints the number of times that vehicle m
    can leave or returns to the depot
    '''
    for m in M:
        if len(N) > len(M):
            model.addCons(quicksum(x[0,j,m] for j in No) >= 1, name=f'leave_depot[{0},{j},{m}]')
        else:
            model.addCons(quicksum(x[0,j,m] for j in No) <= 1, name=f'leave_depot[{0},{j},{m}]')
                       
    '''
    Each customer is visited only once
    '''        
    for j in No:
        model.addCons(quicksum(x[0,j,m] for m in M) == 1, name=f'customer[{i},{j},{m}]')

    '''
    ACTS like a flow conservation constraint:
    Enforces that the vehicle must return to the depot to load
    the demand for the next vertex
    // this one is logically included in the next one
    '''
    # for j in N:
    #     model.addCons((quicksum(x[0,j,m] for m in M if j!= 0)) 
    #               == (quicksum(x[j,0,m] for m in M if j!= 0)), name=f'flow_conservation[{0},{j},{m}]') 
        
    '''
    Enforces that the vehicle must return to the depot from the
    vertex which it delivered // put recharge constraint here
    '''    
    for m in M:
        for j in No:
            model.addCons(x[0,j,m] == x[j,0,m], name=f'sameNode_sameCar[{i},{j},{m}]')
            
                
    #==============================
    # mostrar reunião
    
    '''
    Demand constraints
    '''        
    '''
    Demand Satisfaction // redundant to demand conservation?
    '''
    for j in G:  
        model.addCons(
            quicksum(l[i,j] for i in G if (i,j) in A) == demand[j],
            name=f"demand_satisfaction[{j}]"
        )    
        
    '''
    Vehicle Load Capacity
    ''' 
    for m in M:
        for j in No:  # Assuming No is the set of customers excluding the depot
            model.addCons(
                quicksum(l[i,j] for i in G if (i,j) in A) <= load_capacity[m],
                name=f"vehicle_load_capacity[{m}]"
            )
            
    '''
    Vehicle departs with only demand[i] load, then returns to depot
    Demand Conservation
    '''
    # for m in M:
    #     for j in No:
    #         model.addCons(
    #             quicksum(l[i,j] * x[i,j,m] for i in G if (i,j) in A) -
    #             quicksum(l[j,k] * x[j,k,m] for k in G if (j,k) in A) 
    #             == demand[j], name=f'demand_conservation[{j},{m}]'            
    



    # '''
    # Battery capacity constraints // NÃO TESTEI
    # '''
    # for i in G:
    #     for j in G:
    #         for m in M:
    #             if i != j:
    #                 model.addCons(y[i, j, m] >= 0, name=f"battery_level_nonnegativity[{i},{j},{m}]")
    #                 model.addCons(y[i, j, m] <= battery_capacity, name=f"battery_level_capacity[{i},{j},{m}]")
                    
           
    '''
    Time limit constraint
    '''
    for m in M:
        model.addCons(quicksum(time_cost[i,j]*x[i,j,m] for (i,j) in A if i!=j) <= 8, name=f'timeLimit[{m}]')
    
    # Objective - ONLY TRAVEL TIME and WAITING TIME 
    travel_time_cost = quicksum(input_variables["Ctt"]*time_cost[i,j]*x[i,j,m] for (i,j) in A for m in M)
    model.setObjective(travel_time_cost, "minimize")

    model.optimize()

    model.data = x
    
    return model


In [413]:
if __name__ == "__main__":
    No, N, F, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost = instanSacy(5,3,12000,100)
    model = sacystation(No, N, F, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost)    
    model.optimize()
    
    EPS = 1.e-6  # a small tolerance
    x = model.data  # model.data contains the decision variables

    # Time cost
    # cost = sum(time_cost[i,j]*x[i,j,m] for i,j in A if i!=j)
    # values = cost.terms.values()
    print(f'{distance=}')
    # print("Total time:", sum(values))
    # print(f'Travel time ={(travel_time_cost.terms.values())}')
    # print(f'{time_cost=}')
    # print(f'Total time cost={sum(time_cost.values())}')
    print(f'{load_capacity=}')
    print(f'{demand=}')
    print(N,M, sep='\n')
    
    if model.getStatus() == "optimal":
        print("Optimal value:", model.getObjVal())
    else:
        print("Problem could not be solved to optimality")      

    for (i, j, m) in x:
        if model.getVal(x[i, j, m]) > 0.5:
            print(f"x({i}, {j}, {m}) = {model.getVal(x[i, j, m])}")
        # else:
        #     print(f"x[{i}, {j}, {m}] = 0")
    


distance={(0, 1): 87, (0, 2): 56, (0, 3): 86, (0, 4): 86, (0, 5): 81, (0, 'F1'): 59, (0, 'F2'): 70, (1, 0): 53, (1, 2): 78, (1, 3): 61, (1, 4): 59, (1, 5): 98, (1, 'F1'): 98, (1, 'F2'): 60, (2, 0): 65, (2, 1): 96, (2, 3): 71, (2, 4): 73, (2, 5): 78, (2, 'F1'): 80, (2, 'F2'): 73, (3, 0): 98, (3, 1): 59, (3, 2): 50, (3, 4): 61, (3, 5): 62, (3, 'F1'): 99, (3, 'F2'): 95, (4, 0): 66, (4, 1): 77, (4, 2): 52, (4, 3): 98, (4, 5): 68, (4, 'F1'): 99, (4, 'F2'): 70, (5, 0): 97, (5, 1): 57, (5, 2): 97, (5, 3): 68, (5, 4): 81, (5, 'F1'): 74, (5, 'F2'): 71, ('F1', 0): 60, ('F1', 1): 90, ('F1', 2): 57, ('F1', 3): 85, ('F1', 4): 65, ('F1', 5): 79, ('F1', 'F2'): 74, ('F2', 0): 99, ('F2', 1): 85, ('F2', 2): 78, ('F2', 3): 96, ('F2', 4): 54, ('F2', 5): 59, ('F2', 'F1'): 80}
load_capacity={1: 9073, 2: 9073, 3: 9073}
demand={0: 0, 1: 2140, 2: 3163, 3: 7460, 4: 1447, 5: 1517, 'F1': 0, 'F2': 0}
{0, 1, 2, 3, 4, 5}
[1, 2, 3]
Optimal value: 154.72
x(0, 1, 3) = 1.0
x(0, 2, 1) = 1.0
x(0, 3, 1) = 1.0
x(0, 4, 1) = 

In [414]:
model.writeProblem(r"D:\DAY2DAY\MESTRADO\Codes\SCIP\LPs\problem.lp")
model.freeProb()

wrote problem to file D:\DAY2DAY\MESTRADO\Codes\SCIP\LPs\problem.lp


In [415]:
model = Model("sacystation")

def instanSacy(n_customers, n_vehicles, max_distance):
    No = set(np.arange(1,n_customers+1)) #Set of customers
    N = No | {0}
    F = {0, "F1", "F2"} #Set of recharging stations
    G = N | F
    A = [(i,j) for i,j in product(G,G) if i!=j]
    
    for i in G:     
        demand = {i: (0 if i in F else int(np.random.randint(100, 300, 1)[0])) for i in G} #Demand per customer
        
    M = list(np.arange(1,n_vehicles+1)) #Set of vehicles

    load_capacity = {m:input_variables["Qm"] for m in M} #Load_capacity per vehicle
    
    '''Time cost as a function of distance and avg. speed'''
    distance =  {(i,j):int(np.random.randint(50, max_distance, 25)[0]) for i,j in A}
    time_cost = {(i,j):round((distance[i,j] / input_variables["Avg_V"]),2) for i,j in A} #Travel time cost per arc
    # time_cost = {(i,j):int(np.random.randint(1, max_time_cost, 1)[0]) for i,j in A} 
    
    battery_capacity = input_variables["Battery_capacity_Wh"]

    '''Energy cost as a function of Energy Consumption per Km'''
    EV_avg_energy_consumption = (290+150)/2 #in Wh font:'https://ev-database.org/cheatsheet/energy-consumption-electric-car'
    CE_avg_energy_consumption = (0.16) #Liters/km font:'https://www.smmt.co.uk/wp-content/uploads/sites/2/Heavy-CV-Fuel-Consumption-Fact-Sheet.pdf'
    EV_energy_cost = {(i,j):(EV_avg_energy_consumption/distance[i,j]) for i,j in A} #Energy cost per arc
    CE_energy_cost = {(i,j):(CE_avg_energy_consumption/distance[i,j]) for i,j in A} #Fuel cost per arc
    # energy_cost = {(i,j):int(np.random.randint(1, max_energy_cost, 1)[0]) for i,j in A if j>i} #Energy cost per arc

    
    return No, N, F, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost

No, N, F, G, A, M, demand,load_capacity, battery_capacity, time_cost, distance, EV_energy_cost, CE_energy_cost = instanSacy(2,1,100)

model = Model("sacystation")

#Creating binary decision variable
x = {}
for (i,j,m) in [(i,j,m) for i,j in A for m in M ]:
    x[i,j,m] = model.addVar(vtype="B", name=f"x{(i,j,m)}")
    
L = {}
for (i,j,m) in [(i,j,m) for i,j in A for m in M]:
    L[i,j,m] = model.addVar(vtype="C", name=f"L{(i,j,m)}")    
    
y = {}
for (i,j,m) in [(i,j,m) for i,j in A for m in M]:
    y[i,j,m] = model.addVar(vtype="C", name=f"y{(i,j,m)}")
    
# y2 = {}
# if i!=j:
#     for i, j, m in product(A,M) :
#         y2[i, j, m] = model.addVar(vtype="C", name=f"y2({i},{j},{m})")
 
# print(len([(i,j,m) for i,j,m in product(G,G,M) if i!=j]))    
print(G)    
# print(len(A), y, len(y), sep='\n')

x2 = {}
for (i,j) in A:
    for m in M:
        x2[i,j,m] = model.addVar(vtype="B", name=f"x{(i,j,m)}")
print(len(x),len(x2))
print(No)





{0, 1, 2, 'F1', 'F2'}
20 20
{1, 2}
